In [23]:
import poke_env
import pandas as pd

Je ne sais pas vraiment sur quels types d'encodages partir donc on va en essayer plusieurs.
Ce qui est sûr c'est que je veux prendre en compte les pokémons en questions joués, leurs mooves, leur speed, leurs stats, plus tard leurs objets.

La question c'est comment on représente toutes ces valeurs qui ne sont pas continues ? (id pokémon/type, moovepool)

On va essayer différentes méthodes : 

1) on ne prend pas en compte les ids des pokés, juste leurs types. Pour les mooves, on prend en compte le type et la puissance. Pour représenter le type : one-hot encoding. donc on aura 1 one-hot encoding pour le type du poké, puis 1 pour chacun de ses mooves. Comment relier ça à la table des types ? On va la recréer sous forme de matrice. Emplacement de chaque type (normal = ligne/colonne 1, fire = ligne/colonne 2 etc ..) =

{normal, fire, water, electric, grass, ice, fighting, poison, ground, flying, psychic, bug, rock, ghost, dragon, dark, steel, fairy}

In [24]:
#Table des types, forme matricielle
type_list = [
    "normal", "fire", "water", "electric", "grass", "ice", "fighting",
    "poison", "ground", "flying", "psychic", "bug", "rock", "ghost",
    "dragon", "dark", "steel", "fairy"
]

# Matrice des multiplicateurs d'après le tableau officiel des types Pokémon
type_chart = {
    "normal":   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0, 1, 1, 0.5, 1],
    "fire":     [1, 0.5, 0.5, 1, 2, 2, 1, 1, 1, 1, 1, 2, 0.5, 1, 0.5, 1, 2, 1],
    "water":    [1, 2, 0.5, 1, 0.5, 1, 1, 1, 2, 1, 1, 1, 2, 1, 0.5, 1, 1, 1],
    "electric": [1, 1, 2, 0.5, 0.5, 1, 1, 1, 0, 2, 1, 1, 1, 1, 0.5, 1, 1, 1],
    "grass":    [1, 0.5, 2, 1, 0.5, 1, 1, 0.5, 2, 0.5, 1, 0.5, 2, 1, 0.5, 1, 0.5, 1],
    "ice":      [1, 0.5, 0.5, 1, 2, 0.5, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 0.5, 1],
    "fighting": [2, 1, 1, 1, 1, 2, 1, 0.5, 1, 0.5, 0.5, 0.5, 2, 0, 1, 2, 2, 0.5],
    "poison":   [1, 1, 1, 1, 2, 1, 1, 0.5, 0.5, 1, 1, 1, 0.5, 0.5, 1, 1, 0, 2],
    "ground":   [1, 2, 1, 2, 0.5, 1, 1, 2, 1, 0, 1, 0.5, 2, 1, 1, 1, 2, 1],
    "flying":   [1, 1, 1, 0.5, 2, 1, 2, 1, 1, 1, 1, 2, 0.5, 1, 1, 1, 0.5, 1],
    "psychic":  [1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 0.5, 1, 1, 1, 1, 0, 0.5, 1],
    "bug":      [1, 0.5, 1, 1, 2, 1, 0.5, 0.5, 1, 0.5, 2, 1, 1, 0.5, 1, 2, 0.5, 0.5],
    "rock":     [1, 2, 1, 1, 1, 2, 0.5, 1, 0.5, 2, 1, 2, 1, 1, 1, 1, 0.5, 1],
    "ghost":    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 0.5, 1, 1],
    "dragon":   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0.5, 0],
    "dark":     [1, 1, 1, 1, 1, 1, 0.5, 1, 1, 1, 2, 1, 1, 2, 1, 0.5, 1, 0.5],
    "steel":    [1, 0.5, 0.5, 0.5, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0.5, 2],
    "fairy":    [1, 0.5, 1, 1, 1, 1, 2, 0.5, 1, 1, 1, 1, 1, 1, 2, 2, 0.5, 1],
}

df_type_chart = pd.DataFrame(type_chart, index=type_list).T

import numpy as np

type_list = [
    "normal", "fire", "water", "electric", "grass", "ice", "fighting",
    "poison", "ground", "flying", "psychic", "bug", "rock", "ghost",
    "dragon", "dark", "steel", "fairy"
]

type_to_idx = {t: i for i, t in enumerate(type_list)}

# Convertir le DataFrame précédent en matrice NumPy
type_matrix = df_type_chart.to_numpy()

print(type_matrix)

[[1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.5 0.  1.  1.  0.5 1. ]
 [1.  0.5 0.5 1.  2.  2.  1.  1.  1.  1.  1.  2.  0.5 1.  0.5 1.  2.  1. ]
 [1.  2.  0.5 1.  0.5 1.  1.  1.  2.  1.  1.  1.  2.  1.  0.5 1.  1.  1. ]
 [1.  1.  2.  0.5 0.5 1.  1.  1.  0.  2.  1.  1.  1.  1.  0.5 1.  1.  1. ]
 [1.  0.5 2.  1.  0.5 1.  1.  0.5 2.  0.5 1.  0.5 2.  1.  0.5 1.  0.5 1. ]
 [1.  0.5 0.5 1.  2.  0.5 1.  1.  2.  2.  1.  1.  1.  1.  2.  1.  0.5 1. ]
 [2.  1.  1.  1.  1.  2.  1.  0.5 1.  0.5 0.5 0.5 2.  0.  1.  2.  2.  0.5]
 [1.  1.  1.  1.  2.  1.  1.  0.5 0.5 1.  1.  1.  0.5 0.5 1.  1.  0.  2. ]
 [1.  2.  1.  2.  0.5 1.  1.  2.  1.  0.  1.  0.5 2.  1.  1.  1.  2.  1. ]
 [1.  1.  1.  0.5 2.  1.  2.  1.  1.  1.  1.  2.  0.5 1.  1.  1.  0.5 1. ]
 [1.  1.  1.  1.  1.  1.  2.  2.  1.  1.  0.5 1.  1.  1.  1.  0.  0.5 1. ]
 [1.  0.5 1.  1.  2.  1.  0.5 0.5 1.  0.5 2.  1.  1.  0.5 1.  2.  0.5 0.5]
 [1.  2.  1.  1.  1.  2.  0.5 1.  0.5 2.  1.  2.  1.  1.  1.  1.  0.5 1. ]
 [0.  1.  1.  1.  1.  1. 

Ainsi le vecteur 'type normal' est : [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

type qui attaque = sur la ligne, type qui reçoit = sur la colonne

On prend en compte le type de chaque pokémon, pour que des corrélations puissent se faire sur les switchs. Donc dans mon embedd j'aurais les types des 5 autres pokés.

Maintenant on va tester un premier embedding ! 
On est obligés d'avoir un vecteur 1D donc il va falloir se dire où est-ce qu'on met quoi :

Embedding = ['moves_base_power*moves_dmg_multiplier','pokemon_types']

Rq : ne prendra pas en compte les pokemons sans types (l'attaque feu qui fait perdre le type là, bref...). peut-être créer une 19ème colonne ? colonne No_type genre

In [25]:
#Fonctions nécessaires
def obtain_one_hot_vector_type(p) :
    """renvoie le type du pokemon au format one-hot encoding
    entrée : poke_env.environment.pokemon.Pokemon
    sortie : np.array, de longueur fixée 18"""
    vec = np.zeros(18)

    if p.type_1 : 
        vec[type_to_idx[p.type_1.name.lower()]] = 1.0
    if p.type_2 :
        vec[type_to_idx[p.type_2.name.lower()]] = 1.0
    return vec

def obtain_pokemon_types(battle):

    # Encode ta team
    vectors_my_team = [
        obtain_one_hot_vector_type(p) if not p.fainted else np.zeros(len(type_to_idx), dtype=np.float32)
        for _, p in battle.team.items()
    ]
    while len(vectors_my_team) < 6:
        vectors_my_team.append(np.zeros(len(type_to_idx), dtype=np.float32))
    my_team_type = np.concatenate(vectors_my_team)

    # Encode la team adverse
    vectors_opponent_team = [
        obtain_one_hot_vector_type(p) if not p.fainted else np.zeros(len(type_to_idx), dtype=np.float32)
        for _, p in battle.opponent_team.items()
    ]
    while len(vectors_opponent_team) < 6:
        vectors_opponent_team.append(np.zeros(len(type_to_idx), dtype=np.float32))
    opponent_team_type = np.concatenate(vectors_opponent_team)

    # Concatène les deux parties
    pokemon_types = np.concatenate([my_team_type, opponent_team_type])
    
    return pokemon_types


In [26]:
import numpy as np
from gymnasium.spaces import Space, Box
from poke_env.player import Gen8EnvSinglePlayer
from poke_env.data import GenData
import numpy as np
from poke_env.environment.abstract_battle import AbstractBattle
import torch
from gymnasium.spaces import Discrete, Box

# Initialiser GenData pour la génération souhaitée (par exemple, génération 8)
gen_data = GenData.from_gen(8)

# Accéder au tableau des types
type_chart = gen_data.type_chart


class embedding_Player(Gen8EnvSinglePlayer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.action_space = Discrete(9)  # ✅ attribut classique
    
    #Toujours mêmes valeurs de reward
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]


        #Pokemon types  
        pokemon_types = obtain_pokemon_types(battle)

        # Final vector with 10 components
        final_vector = np.concatenate(
            [
                moves_real_power,
                pokemon_types,
            ]
        )

        return np.float32(final_vector)

    def describe_embedding(self) -> Space:
        low = (
            [-1] * 4 +          # real power
            [0] * 108 +         # my team types
            [0] * 108           # opponent team types
        )
        high = (
            [3] * 4 +           # real power
            [1] * 108 +         # my team types
            [1] * 108           # opponent team types
        )

        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32
        )
    
    def action_to_move(self, action: int, battle: AbstractBattle):
        order = super().action_to_move(action, battle)
        order.dynamax = False  # 🔥 désactive Dynamax pour toutes les actions
        return order



In [27]:
##DEBUG
from poke_env.environment.pokemon import Pokemon

# TEST obtain_one_hot_vector_type(p) ###
p = Pokemon(gen=8, species="abomasnow")
#print(type(p))
#print(p.species)    # 'abomasnow'
#print((p.type_1.name.lower()))   # 'grass'
#print(p.type_2)

vec = np.zeros(18)

if p.type_1 : 
    vec[type_to_idx[p.type_1.name.lower()]] = 1.0
if p.type_2 :
    vec[type_to_idx[p.type_2.name.lower()]] = 1.0

#print(vec)


# TEST obtain_pokemon_types(battle) ###



In [28]:
from poke_env.player.random_player import RandomPlayer

class NoDynamaxRandomPlayer(RandomPlayer):
    def choose_move(self, battle):
        choice = super().choose_move(battle)
        if choice.dynamax:
            choice.dynamax = False  # désactive l'option
        return choice

***TRAINING***

In [29]:

opponent = NoDynamaxRandomPlayer(battle_format="gen8randombattle")
from stable_baselines3.common.vec_env import DummyVecEnv

# instanciation du player
train_env_raw = embedding_Player(
    battle_format="gen8randombattle",
    opponent=RandomPlayer(battle_format="gen8randombattle"),
    start_challenging=True
)

# wrap dans DummyVecEnv (SB3 attend un vecteur d’envs, même pour un seul)
train_env = DummyVecEnv([lambda: train_env_raw])


In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#Réseau perceptron à une couche cachée, sortie linéaire, f activation = Relu,
class DQNModel(nn.Module):
    def __init__(self, input_dim, n_actions):
        super(DQNModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)
        self.layer2 = nn.Linear(128, 64)
        self.out = nn.Linear(64, n_actions)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.out(x)


In [31]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env

# Si ton env est déjà Gym-compatible (comme avec poke-env Gym wrapper)
env = train_env

model = DQN(
    policy="MlpPolicy",
    env=env,
    learning_rate=2.5e-4,
    buffer_size=10000,
    learning_starts=1000,
    batch_size=32,
    gamma=0.5,
    train_freq=1,
    target_update_interval=1,
    exploration_fraction=1.0,
    exploration_final_eps=0.05,
    policy_kwargs=dict(activation_fn=nn.ReLU, net_arch=[128, 64])
)



In [32]:
from tqdm.notebook import tqdm
from stable_baselines3.common.callbacks import BaseCallback
from IPython.display import display
import ipywidgets as widgets

class CustomTQDMCallback(BaseCallback):
    def __init__(self, total_timesteps, check_freq=500, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.total_timesteps = total_timesteps
        self.progress_bar = None

    def _on_training_start(self) -> None:
        self.progress_bar = tqdm(total=self.total_timesteps, desc="📊 Training progress")
    
    def _on_step(self) -> bool:
        # Avance la barre
        if self.progress_bar:
            self.progress_bar.update(1)

        # Affichage toutes les X étapes
        if self.n_calls % self.check_freq == 0 and self.verbose:
            self.progress_bar.set_postfix_str(f"Timesteps: {self.num_timesteps}")
        
        return True

    def _on_training_end(self) -> None:
        if self.progress_bar:
            self.progress_bar.close()



In [45]:
steps = 50000
callback = CustomTQDMCallback(total_timesteps = steps, check_freq=1000, verbose=1)
model.learn(
    total_timesteps=steps,
    callback=callback
)
model.save("embedding_1")

📊 Training progress:   0%|          | 0/50000 [00:00<?, ?it/s]

***TEST***

In [46]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env

class EmbeddingTestPlayer(Gen8EnvSinglePlayer):

    def __init__(self, model_path="embedding_1", **kwargs):
        super().__init__(**kwargs)
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print(f"📥 Modèle chargé depuis {model_path}")

        #Toujours mêmes valeurs de reward
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]


        #Pokemon types  
        pokemon_types = obtain_pokemon_types(battle)

        # Final vector with 10 components
        final_vector = np.concatenate(
            [
                moves_real_power,
                pokemon_types,
            ]
        )

        return np.float32(final_vector)

    def describe_embedding(self) -> Space:
        low = (
            [-1] * 4 +          # real power
            [0] * 108 +         # my team types
            [0] * 108           # opponent team types
        )
        high = (
            [3] * 4 +           # real power
            [1] * 108 +         # my team types
            [1] * 108           # opponent team types
        )

        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32
        )
    
    def action_to_move(self, action, battle):
        moves = battle.available_moves
        switches = battle.available_switches
        total_actions = len(moves) + len(switches)

        #print(f"🔢 DQN → action={action} | #moves={len(moves)} | #switches={len(switches)} | total={total_actions}")

        if 0 <= action < len(moves):
            return self.create_order(moves[action])
        elif len(moves) <= action < total_actions:
            return self.create_order(switches[action - len(moves)])
        else:
            #print("❌ Action hors bornes ! Fallback sur move aléatoire")
            return self.choose_random_move(battle)
        
    def predict(self, obs):
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        q_values = self.model.q_net(obs_tensor)
        print(f"📊 Q-values : {q_values.detach().numpy().flatten()}")
        action = int(torch.argmax(q_values).item())
        return action

In [47]:
opponent = NoDynamaxRandomPlayer(battle_format="gen8randombattle")
eval_env = EmbeddingTestPlayer(
    battle_format="gen8randombattle", opponent=opponent, start_challenging=True
)

📥 Modèle chargé depuis embedding_1


In [50]:
n_eval_episodes = 100
rewards = []
wins = 0

obs, _ = eval_env.reset()
for _ in tqdm(range(n_eval_episodes)):
    done = False
    total_reward = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, _ = eval_env.step(action)
        done = terminated or truncated

        total_reward += reward
        if done and reward > 0:
            wins += 1

    rewards.append(total_reward)
    obs, _ = eval_env.reset()

print(f"✅ {wins} victoires sur {n_eval_episodes} matchs")
print(f"🎯 Reward moyen : {sum(rewards) / len(rewards):.2f}")



  0%|          | 0/100 [00:00<?, ?it/s]

✅ 53 victoires sur 100 matchs
🎯 Reward moyen : 2.27


2) Embedding = ['moves_base_power*moves_dmg_multiplier','pokemon_types','pokemon_stats']

3) On va rajouter les talents d'immunité. On va rajouter un case pour les talents qui donnent des immunités, voila on fait ça à la mano pas grave (if ability in list_immunies_abilities ...)

In [49]:
#VS me
from DQ_simple import DQ_simple 
class DQ_simpleemb1(DQ_simple) :
    def __init__(self, model_path = "embedding_1", battle_format="gen8randombattle"):
        super().__init__(battle_format=battle_format)
        
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print("📥 Modèle DQN chargé :", self.model)
    
    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
            moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        #Pokemon types  
        pokemon_types = obtain_pokemon_types(battle)

        # Final vector with 10 components
        final_vector = np.concatenate(
            [
                moves_real_power,
                pokemon_types,
            ]
        )

        return np.float32(final_vector)

    def choose_move(self, battle):
        print("👉 choose_move appelée !")
        # 🔍 Debug : Voir les moves disponibles
        print(f"🔍 Moves disponibles : {[move.id for move in battle.available_moves]}")

        # Obtenir l'observation de l'état du combat
        obs = self.embed_battle(battle)
        print("📊 Observation de l'état :", obs)

        # Transformer en format PyTorch
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        print("📊 Tensor pour le modèle :", obs_tensor)

        # Prédire l'action avec le modèle DQN
        action = int(self.model.predict(obs_tensor, deterministic=True)[0])
        print("🎯 Action choisie par DQN :", action)
        if 0 <= action < len(battle.available_moves):
            move = battle.available_moves[action]
            print(f"✅ Move choisi : {move.id}")

            order = self.create_order(move)
            print(f"📤 Ordre créé : {order}")  # Debug : voir l'ordre exact généré

            return order
        else:
            print(f"❌ Action {action} invalide, on joue un move aléatoire !")
            return self.choose_random_move(battle)